<a href="https://colab.research.google.com/github/Naidala/CNN_medical_imaging/blob/main/Task3bis_multiLabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
EPOCHS = 75
INIT_LR = 1e-3
BS = 64
IMAGE_DIMS = (150, 150, 1)

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
from google.colab import drive
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import copy


############################

def plot_acc_loss(history):
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(len(acc))

  plt.plot(epochs, acc, 'r', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.figure()
  plt.plot(epochs, loss, 'r', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()


def best_epoch(history):
  x = history.history['val_loss']
  k = 3
  trend = []
  stop = 0
  for i in range(k,len(x)-k):
    if stop == 0:
      num = 0
      for j in range(-k, k+1):
        num = x[i+j] + num
      if len(trend) == 0:
        trend.append(num/(2*k+1))
      elif num/(2*k+1) <= trend[len(trend)-1] + 0.005:
        trend.append(num/(2*k+1))
      else:
        stop = 1
  print(2*k+len(trend))


############################
drive.mount('/content/drive/')
out_path = "/content/drive/My Drive/CompIntProjects/FinalProject2019/numpy data/"

def load_data():
  train_images = np.load(os.path.join(out_path,'train_tensor.npy'))
  train_labels = np.load(os.path.join(out_path,'train_labels.npy'))
  test_images = np.load(os.path.join(out_path,'public_test_tensor.npy'))
  test_labels = np.load(os.path.join(out_path,'public_test_labels.npy'))
  train_images = train_images.astype('float32') / 65535   #BW 16-bit
  train_images = train_images.reshape((len(train_images), 150, 150, 1))
  test_images = test_images.astype('float32') / 65535
  test_images = test_images.reshape((len(test_images), 150, 150, 1))
  return train_images, train_labels, test_images, test_labels


def abnormal_variables():
  train_images, train_labels, test_images, test_labels = load_data()
  train_images = train_images[1:len(train_images):2]
  train_labels = train_labels[1:len(train_labels):2]
  test_images = test_images[1:len(test_images):2]
  test_labels = test_labels[1:len(test_labels):2]
  return train_images, train_labels, test_images, test_labels


def labels_in_four(labels):
  label_words = []
  for i in range(0,len(labels)):
    if labels[i] == 1:
      label_words.append(('mass','good'))
    elif labels[i] == 2:
      label_words.append(('mass','bad'))
    elif labels[i] == 3:
      label_words.append(('calc','good'))
    else:
      label_words.append(('calc','bad'))
  return label_words


def augment(train_images, train_labels):
  np.random.seed(5)
  np.random.shuffle(train_images)
  np.random.seed(5)
  np.random.shuffle(train_labels)
  PERCENTAGE_VALIDATION = 0.2
  num_validation_samples = round(len(train_labels)*PERCENTAGE_VALIDATION)
  my_val_images = copy.deepcopy(train_images[:num_validation_samples])
  my_val_labels = copy.deepcopy(train_labels[:num_validation_samples])
  my_train_images = copy.deepcopy(train_images[num_validation_samples:])
  my_train_labels = copy.deepcopy(train_labels[num_validation_samples:])
  return my_train_images, my_train_labels, my_val_images, my_val_labels

TensorFlow 1.x selected.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense

class SmallerVGGNet:			# simpler versione of VGGNet
	def build(classes, finalAct="softmax"):
		model = Sequential()

		model.add(Conv2D(32, (3, 3), padding="same",input_shape=IMAGE_DIMS))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(3, 3)))
		model.add(Dropout(0.25))

		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(3, 3)))
		model.add(Dropout(0.25))

		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		model.add(Flatten())
		model.add(Dense(150))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.45))

		model.add(Dense(classes))
		model.add(Activation(finalAct))

		return model

Using TensorFlow backend.


In [ ]:
train_images_4, train_labels_4, test_images_4, test_labels_4 = abnormal_variables()
train_labels_4 = labels_in_four(train_labels_4)
test_labels_4 = labels_in_four(test_labels_4)

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()   # library to transform the word tuple (e.g. ('mass','bad'))
mlb.fit(train_labels_4)       # into a binary vector [1, 0, 0, 1]
mlb.fit(test_labels_4)
print(mlb.classes_)
train_labels_4 = mlb.transform(train_labels_4)
test_labels_4 = mlb.transform(test_labels_4)

my_train_images, my_train_labels, my_val_images, my_val_labels = augment(train_images_4, train_labels_4)

['bad' 'calc' 'good' 'mass']


In [ ]:
aug = ImageDataGenerator(rotation_range = 20,
                         shear_range = 0.3,
                         zoom_range = 0.5,
                         height_shift_range = 0.1,
                         width_shift_range = 0.1,
                         horizontal_flip = True,
                         vertical_flip = True,
                         fill_mode = 'nearest')

In [ ]:
from keras.optimizers import Adam
model = SmallerVGGNet.build(
	classes=len(mlb.classes_),
	finalAct="sigmoid")
#opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)		# raggiunge 0.71 e non 0.74, probabilmente migliore per epoche lunghe (es. 1000)?
opt = Adam(lr=INIT_LR)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

H = model.fit_generator(
	aug.flow(my_train_images,my_train_labels,batch_size = BS),
  validation_data = (my_val_images,my_val_labels),
	steps_per_epoch = len(my_train_labels) // BS,
	epochs=EPOCHS,
  verbose=1)

Epoch 1/75
33/33 [==============================] - 4s 118ms/step - loss: 0.6936 - accuracy: 0.5651 - val_loss: 0.6858 - val_accuracy: 0.5748
Epoch 2/75
33/33 [==============================] - 4s 107ms/step - loss: 0.6584 - accuracy: 0.6123 - val_loss: 0.6817 - val_accuracy: 0.5748
Epoch 3/75
33/33 [==============================] - 4s 109ms/step - loss: 0.6440 - accuracy: 0.6466 - val_loss: 0.6778 - val_accuracy: 0.5710
Epoch 4/75
33/33 [==============================] - 4s 110ms/step - loss: 0.6440 - accuracy: 0.6409 - val_loss: 0.6804 - val_accuracy: 0.5748
Epoch 5/75
33/33 [==============================] - 4s 108ms/step - loss: 0.6474 - accuracy: 0.6312 - val_loss: 0.6732 - val_accuracy: 0.5729
Epoch 6/75
33/33 [==============================] - 4s 110ms/step - loss: 0.6385 - accuracy: 0.6517 - val_loss: 0.6720 - val_accuracy: 0.5710
Epoch 7/75
33/33 [==============================] - 4s 109ms/step - loss: 0.6398 - accuracy: 0.6443 - val_loss: 0.6629 - val_accuracy: 0.5710
Epoch 

In [ ]:
test_loss, test_acc = model.evaluate(test_images_4, test_labels_4)
print('test_acc: ', test_acc)

336/336 [==============================] - 0s 258us/step
test_acc:  0.7395833134651184


In [ ]:
#save_dir = "/content/drive/My Drive/CompIntProjects/4classes"
#if not os.path.exists(save_dir):
#  os.makedirs(save_dir)
#model.save(os.path.join(save_dir,'4classes_multiLabel.h5'))

In [ ]:
from tensorflow.keras.models import load_model
save_dir = "/content/drive/My Drive/CompIntProjects/4classes"
model = load_model(os.path.join(save_dir,"4classes_multiLabel.h5"))
test_loss, test_acc = model.evaluate(test_images_4, test_labels_4)
print('test_acc: ', test_acc)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
336/336 [==============================] - 2s 5ms/sample - loss: 0.4942 - acc: 0.7574
test_acc:  0.75744045


In [ ]:
# confronto tra multiLabel
prediction = model.predict(test_images_4)
prediction = np.round(prediction)

count = 0
for i in range(0,len(test_labels_4)):
    if (test_labels_4[i] == prediction[i]).all():
      count = count +1
print(count/len(test_labels_4))  # 0.56 as in 0.75^2, since accuracy for binary classification
                                    # like bernoulli independent variable

0.5595238095238095
